In [74]:
import pandas as pd
import sqlite3
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, mean_poisson_deviance, mean_gamma_deviance
from sklearn.metrics import mean_squared_error, mean_absolute_error, classification_report
import xgboost as xgb

conn = sqlite3.connect(r'C:\Users\Owner\dev\algobetting\infra\data\db\algobetting.db')

df = pd.read_sql_query("""
                    SELECT DISTINCT
                        f.*,
                        ms.passing_corner_kicks as corner_kicks,
                        ms.opp_passing_corner_kicks as opp_corner_kicks
                    FROM 
                        team_all_features_365_005 f
                    JOIN
                        fbref_match_summary_v2 ms
                            ON ms.match_url = f.match_url
                            AND ms.team = f.team
                    WHERE 
                        f.team_rolling_summary_minutes IS NOT NULL
                        AND NOT (f.`is_promoted?` = 1 AND f.`is_early_season?` = 1)
                       """, conn)

df

,match_url,match_date,season,division,team,opp_team,is_home,gw,is_promoted?,is_early_season?,...,opp_team_rolling_conceded_defense_challenge_tackles_pct,opp_team_rolling_conceded_defense_challenges_lost,opp_team_rolling_conceded_defense_blocked_shots,opp_team_rolling_conceded_defense_blocked_passes,opp_team_rolling_conceded_defense_tackles_interceptions,opp_team_rolling_conceded_defense_clearances,opp_team_rolling_conceded_defense_errors,opp_team_rolling_conceded_keeper_psxg,corner_kicks,opp_corner_kicks
0,https://fbref.com/en/matches/e4bb1c35/Tottenha...,2025-05-25 00:00:00,2024-2025,Premier League,Tottenham,Brighton,1,38,0,0,...,50.720617,8.796946,3.393454,7.020603,26.403010,27.177737,0.862971,1.299603,2.0,11.0
1,https://fbref.com/en/matches/e4bb1c35/Tottenha...,2025-05-25 00:00:00,2024-2025,Premier League,Brighton,Tottenham,0,38,0,0,...,57.889854,7.949406,3.378971,9.378588,32.038948,23.558919,0.911136,1.805191,11.0,2.0
2,https://fbref.com/en/matches/1ff370e8/Bournemo...,2025-05-25 00:00:00,2024-2025,Premier League,Bournemouth,Leicester City,1,38,0,0,...,53.893970,6.606014,2.506894,6.904780,22.901258,21.697324,0.636969,1.811285,6.0,1.0
3,https://fbref.com/en/matches/1ff370e8/Bournemo...,2025-05-25 00:00:00,2024-2025,Premier League,Leicester City,Bournemouth,0,38,1,0,...,49.279905,7.520887,4.153870,6.746572,23.732322,26.770789,1.451179,1.460762,1.0,6.0
4,https://fbref.com/en/matches/36844e73/Newcastl...,2025-05-25 00:00:00,2024-2025,Premier League,Newcastle Utd,Everton,1,38,0,0,...,53.064874,6.895930,2.600829,6.075265,24.063503,26.142993,0.567784,1.251694,12.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39901,https://fbref.com/en/matches/e8e75a75/Como-Cag...,2022-08-13 00:00:00,2022-2023,Serie B,Como,Cagliari,1,1,0,1,...,47.905175,6.408071,3.518286,6.676135,21.826232,17.942850,0.219500,1.367898,6.0,5.0
39902,https://fbref.com/en/matches/e8e75a75/Como-Cag...,2022-08-13 00:00:00,2022-2023,Serie B,Cagliari,Como,0,1,0,1,...,42.604852,8.589310,2.172112,6.611274,23.664988,14.871051,0.075024,1.541054,5.0,6.0
39903,https://fbref.com/en/matches/99979e25/Cittadel...,2022-08-13 00:00:00,2022-2023,Serie B,Cittadella,Pisa,1,1,0,1,...,47.089937,7.713670,2.909231,6.588752,27.403947,19.741195,0.237578,1.160446,5.0,5.0
39904,https://fbref.com/en/matches/99979e25/Cittadel...,2022-08-13 00:00:00,2022-2023,Serie B,Pisa,Cittadella,0,1,0,1,...,50.507986,5.815970,3.866474,9.173457,27.363966,20.635355,0.149176,0.957865,5.0,5.0


In [75]:
final_df = df

In [76]:
final_df["outcome_over"] = np.where(final_df["corner_kicks"] > 5.5, 1, 0)

final_df

,match_url,match_date,season,division,team,opp_team,is_home,gw,is_promoted?,is_early_season?,...,opp_team_rolling_conceded_defense_challenges_lost,opp_team_rolling_conceded_defense_blocked_shots,opp_team_rolling_conceded_defense_blocked_passes,opp_team_rolling_conceded_defense_tackles_interceptions,opp_team_rolling_conceded_defense_clearances,opp_team_rolling_conceded_defense_errors,opp_team_rolling_conceded_keeper_psxg,corner_kicks,opp_corner_kicks,outcome_over
0,https://fbref.com/en/matches/e4bb1c35/Tottenha...,2025-05-25 00:00:00,2024-2025,Premier League,Tottenham,Brighton,1,38,0,0,...,8.796946,3.393454,7.020603,26.403010,27.177737,0.862971,1.299603,2.0,11.0,0
1,https://fbref.com/en/matches/e4bb1c35/Tottenha...,2025-05-25 00:00:00,2024-2025,Premier League,Brighton,Tottenham,0,38,0,0,...,7.949406,3.378971,9.378588,32.038948,23.558919,0.911136,1.805191,11.0,2.0,1
2,https://fbref.com/en/matches/1ff370e8/Bournemo...,2025-05-25 00:00:00,2024-2025,Premier League,Bournemouth,Leicester City,1,38,0,0,...,6.606014,2.506894,6.904780,22.901258,21.697324,0.636969,1.811285,6.0,1.0,1
3,https://fbref.com/en/matches/1ff370e8/Bournemo...,2025-05-25 00:00:00,2024-2025,Premier League,Leicester City,Bournemouth,0,38,1,0,...,7.520887,4.153870,6.746572,23.732322,26.770789,1.451179,1.460762,1.0,6.0,0
4,https://fbref.com/en/matches/36844e73/Newcastl...,2025-05-25 00:00:00,2024-2025,Premier League,Newcastle Utd,Everton,1,38,0,0,...,6.895930,2.600829,6.075265,24.063503,26.142993,0.567784,1.251694,12.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39901,https://fbref.com/en/matches/e8e75a75/Como-Cag...,2022-08-13 00:00:00,2022-2023,Serie B,Como,Cagliari,1,1,0,1,...,6.408071,3.518286,6.676135,21.826232,17.942850,0.219500,1.367898,6.0,5.0,1
39902,https://fbref.com/en/matches/e8e75a75/Como-Cag...,2022-08-13 00:00:00,2022-2023,Serie B,Cagliari,Como,0,1,0,1,...,8.589310,2.172112,6.611274,23.664988,14.871051,0.075024,1.541054,5.0,6.0,0
39903,https://fbref.com/en/matches/99979e25/Cittadel...,2022-08-13 00:00:00,2022-2023,Serie B,Cittadella,Pisa,1,1,0,1,...,7.713670,2.909231,6.588752,27.403947,19.741195,0.237578,1.160446,5.0,5.0,0
39904,https://fbref.com/en/matches/99979e25/Cittadel...,2022-08-13 00:00:00,2022-2023,Serie B,Pisa,Cittadella,0,1,0,1,...,5.815970,3.866474,9.173457,27.363966,20.635355,0.149176,0.957865,5.0,5.0,0


In [77]:
X = final_df.drop(columns=["team", "opp_team", "match_url", "match_date", "corner_kicks", "opp_corner_kicks", "outcome_over"])
# Use dummies for nominal categorical features
X = pd.get_dummies(X, columns=["season", "division"], drop_first=True)
y = final_df["outcome_over"]

# Use matches after a certain date as test
cutoff_date = '2024-08-01'  # adjust as needed
final_df['match_date'] = pd.to_datetime(final_df['match_date'])

train_mask = final_df['match_date'] < cutoff_date
test_mask = final_df['match_date'] >= cutoff_date

X_train = X[train_mask]
X_test = X[test_mask]
y_train = y[train_mask]
y_test = y[test_mask]  

In [78]:
# Train XGBoost classifier
model = xgb.XGBClassifier(enable_categorical=True, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.626

Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.80      0.73      3366
           1       0.46      0.32      0.38      1856

    accuracy                           0.63      5222
   macro avg       0.57      0.56      0.55      5222
weighted avg       0.60      0.63      0.61      5222



In [79]:
# Get feature importance as text
feature_importance = model.get_booster().get_score(importance_type='weight')
sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)

print("Feature Importance (by weight):")
print("-" * 40)
for feature, importance in sorted_features:
    print(f"{feature:<25} {importance:>10}")

Feature Importance (by weight):
----------------------------------------
opp_team_rolling_conceded_defense_errors       39.0
opp_team_rolling_defense_errors       38.0
opp_team_rolling_conceded_defense_tackles_def_3rd       36.0
team_rolling_pass_types_corner_kicks       35.0
team_rolling_conceded_defense_errors       35.0
opp_team_rolling_conceded_possession_touches_def_pen_area       35.0
opp_team_rolling_conceded_pass_types_corner_kicks       35.0
opp_team_rolling_misc_ball_recoveries       34.0
opp_team_rolling_conceded_possession_miscontrols       34.0
team_rolling_pass_types_corner_kicks_out       32.0
team_rolling_conceded_pass_types_through_balls       32.0
opp_team_rolling_defense_blocked_shots       32.0
opp_team_rolling_conceded_defense_tackles_att_3rd       32.0
team_rolling_possession_miscontrols       31.0
team_rolling_defense_tackles_att_3rd       31.0
opp_team_rolling_summary_minutes       31.0
opp_team_rolling_misc_cards_yellow_red       31.0
opp_team_rolling_conceded_

In [80]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
import pickle
import os
import optuna
import numpy as np

# Quick toggle - set to True to run search, False to load existing model
RUN_OPTUNA_SEARCH = True  # Change to False if you want to load existing model

model_file = 'best_xgb_corner_model_optuna.pkl'

def objective(trial):
    # Define VERY STRICT parameter search space to prevent overfitting
    params = {
        'random_state': 42,
        # STRICT REGULARIZATION RANGES
        'max_depth': trial.suggest_int('max_depth', 2, 4),  # Very shallow trees
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.08),  # Lower learning rates
        'n_estimators': trial.suggest_int('n_estimators', 200, 600),  # More trees, slower learning
        'reg_alpha': trial.suggest_float('reg_alpha', 1.0, 10.0),  # Strong L1 regularization
        'reg_lambda': trial.suggest_float('reg_lambda', 3.0, 15.0),  # Very strong L2 regularization
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 0.7),  # Heavy feature sampling
        'subsample': trial.suggest_float('subsample', 0.6, 0.8),  # Row sampling
        'min_child_weight': trial.suggest_int('min_child_weight', 5, 20),  # Large minimum samples
        'gamma': trial.suggest_float('gamma', 0.1, 2.0),  # Minimum split loss
    }
    
    # Create model for classification (over/under corners)
    # If you want to predict exact corner count, change to XGBRegressor with objective='count:poisson'
    model = xgb.XGBClassifier(**params)
    
    # Use cross-validation for evaluation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
    
    return cv_scores.mean()

if RUN_OPTUNA_SEARCH or not os.path.exists(model_file):
    print("Running Optuna optimization with STRICT regularization for corner prediction...")
    print("Parameter ranges designed to prevent overfitting:")
    print("- max_depth: 2-4 (very shallow)")
    print("- reg_alpha: 1.0-10.0 (strong L1)")
    print("- reg_lambda: 3.0-15.0 (very strong L2)")
    print("- colsample_bytree: 0.3-0.7 (heavy feature sampling)")
    print("- min_child_weight: 5-20 (large minimum samples)")
    
    # Create study
    study = optuna.create_study(direction='maximize')
    
    # Optimize
    study.optimize(objective, n_trials=100)  # More trials since we want best regularization
    
    print(f"\nBest trial: {study.best_trial.number}")
    print(f"Best value (accuracy): {study.best_value:.4f}")
    print(f"Best params: {study.best_params}")
    
    # Check if parameters are appropriately strict
    best_params = study.best_params
    print(f"\nRegularization strength check:")
    print(f"- max_depth: {best_params['max_depth']} (good if ≤4)")
    print(f"- reg_lambda: {best_params['reg_lambda']:.2f} (good if ≥5)")
    print(f"- colsample_bytree: {best_params['colsample_bytree']:.2f} (good if ≤0.6)")
    print(f"- min_child_weight: {best_params['min_child_weight']} (good if ≥5)")
    
    # Train final model with best parameters
    model = xgb.XGBClassifier(**best_params, random_state=42)
    model.fit(X_train, y_train)
    
    # Save model
    with open(model_file, 'wb') as f:
        pickle.dump(model, f)
    
    print(f"\nModel saved to {model_file}")
    
else:
    print(f"Loading existing model from {model_file}")
    with open(model_file, 'rb') as f:
        model = pickle.load(f)

# Make predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)

print(f"\nFinal Model Performance:")
print(f"Accuracy: {accuracy:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred))

# Show final model parameters
print(f"\nFinal model parameters:")
final_params = model.get_params()
key_params = ['max_depth', 'learning_rate', 'n_estimators', 'reg_alpha', 'reg_lambda', 
              'colsample_bytree', 'subsample', 'min_child_weight', 'gamma']

for param in key_params:
    if param in final_params:
        value = final_params[param]
        print(f"{param}: {value}")

# Feature importance (top 15 only to avoid clutter)
if hasattr(model, 'feature_importances_'):
    feature_importance = model.feature_importances_
    feature_names = X_train.columns
    
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 15 Feature Importances:")
    print("=" * 60)
    for i, (_, row) in enumerate(importance_df.head(15).iterrows()):
        print(f"{i+1:2d}. {row['feature']:<40} {row['importance']:.4f}")


[I 2025-07-15 09:05:59,861] A new study created in memory with name: no-name-59f7c89e-5938-4036-84fe-0a281adc3943


Running Optuna optimization with STRICT regularization for corner prediction...
Parameter ranges designed to prevent overfitting:
- max_depth: 2-4 (very shallow)
- reg_alpha: 1.0-10.0 (strong L1)
- reg_lambda: 3.0-15.0 (very strong L2)
- colsample_bytree: 0.3-0.7 (heavy feature sampling)
- min_child_weight: 5-20 (large minimum samples)


[I 2025-07-15 09:06:28,075] Trial 0 finished with value: 0.5472003343986137 and parameters: {'max_depth': 2, 'learning_rate': 0.04588918235998516, 'n_estimators': 567, 'reg_alpha': 7.595758315179402, 'reg_lambda': 6.463174367333435, 'colsample_bytree': 0.6874085663162304, 'subsample': 0.7075167564314978, 'min_child_weight': 5, 'gamma': 1.376343220884662}. Best is trial 0 with value: 0.5472003343986137.
[I 2025-07-15 09:06:51,118] Trial 1 finished with value: 0.5387527581816842 and parameters: {'max_depth': 3, 'learning_rate': 0.06581853524010887, 'n_estimators': 341, 'reg_alpha': 5.404302612509861, 'reg_lambda': 10.011224402409443, 'colsample_bytree': 0.3256933839126118, 'subsample': 0.6177591940030139, 'min_child_weight': 11, 'gamma': 1.4496175984558357}. Best is trial 0 with value: 0.5472003343986137.
[I 2025-07-15 09:07:16,934] Trial 2 finished with value: 0.5462487856185984 and parameters: {'max_depth': 2, 'learning_rate': 0.06906595680979259, 'n_estimators': 581, 'reg_alpha': 9.23


Best trial: 99
Best value (accuracy): 0.6357
Best params: {'max_depth': 2, 'learning_rate': 0.011885397128919208, 'n_estimators': 229, 'reg_alpha': 1.684695053879709, 'reg_lambda': 14.374285606775594, 'colsample_bytree': 0.5050604458436364, 'subsample': 0.6038169930457393, 'min_child_weight': 19, 'gamma': 0.35316750183252943}

Regularization strength check:
- max_depth: 2 (good if ≤4)
- reg_lambda: 14.37 (good if ≥5)
- colsample_bytree: 0.51 (good if ≤0.6)
- min_child_weight: 19 (good if ≥5)

Model saved to best_xgb_corner_model_optuna.pkl

Final Model Performance:
Accuracy: 0.6637

Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.94      0.78      3366
           1       0.61      0.16      0.25      1856

    accuracy                           0.66      5222
   macro avg       0.64      0.55      0.52      5222
weighted avg       0.65      0.66      0.59      5222


Final model parameters:
max_depth: 2
learning_rate: 0.0118

In [81]:
# Get predictions and probabilities for test set
test_predictions = model.predict(X_test)
test_proba = model.predict_proba(X_test)

# IMPORTANT: Check what the model's classes are
print("Model classes:", model.classes_)
print("Class 0:", model.classes_[0])
print("Class 1:", model.classes_[1])

# Since classes are [0, 1], we need to figure out which one corresponds to 'over' and 'under'
# Let's check what the original y_train looked like
print("\nOriginal y_train unique values:")
print(y_train.unique())

# If y_train was encoded, we need to check the mapping
# Common encodings: 0='over', 1='under' OR 0='under', 1='over'
# Let's check by looking at a few examples from the training data

# Now we know: 0 = 'under', 1 = 'over'
print("\nMapping:")
print("0 = 'under'")
print("1 = 'over'")

over_class_index = 1  # 'over' is class 1
under_class_index = 0  # 'under' is class 0

print(f"'over' is at index: {over_class_index}")
print(f"'under' is at index: {under_class_index}")

# Create dataframe with test set results
test_df = final_df[["match_url", "match_date", "season", "division", 
                    "team", "opp_team", "is_home", "gw", "is_promoted?", "is_early_season?",
                    "corner_kicks"]].loc[X_test.index].copy()

# Create the "over" rows with CORRECT probability assignment
over_df = test_df.copy()
over_df['outcome'] = 'over'
over_df['predicted_is_won'] = (test_predictions == 1).astype(int)  # 1 = 'over'
over_df['prob_win'] = test_proba[:, over_class_index]  # Column 1 = 'over' probability

# Create the "under" rows with CORRECT probability assignment
under_df = test_df.copy()
under_df['outcome'] = 'under'
under_df['predicted_is_won'] = (test_predictions == 0).astype(int)  # 0 = 'under'
under_df['prob_win'] = test_proba[:, under_class_index]  # Column 0 = 'under' probability

# Combine both
double_test_df = pd.concat([over_df, under_df], ignore_index=True)

# Sort by match info and outcome for better readability
double_test_df = double_test_df.sort_values(['match_url', 'outcome'])

print("\nSample of double rows:")
print(double_test_df[['match_url', 'team', 'outcome', 'predicted_is_won', 'prob_win']].head(10))

# Verify probabilities sum to 1 for each team/line combination
print("\nVerification - probabilities should sum to 1.0 for each team:")
team_verification = double_test_df.groupby(['match_url', 'team'])['prob_win'].sum()
print(f"Min sum: {team_verification.min():.6f}")
print(f"Max sum: {team_verification.max():.6f}")
print(f"Mean sum: {team_verification.mean():.6f}")

# Double-check with a specific example
print("\nDouble-check example:")
sample_match = double_test_df['match_url'].iloc[0]
sample_team = double_test_df['team'].iloc[0]

example = double_test_df[
    (double_test_df['match_url'] == sample_match) & 
    (double_test_df['team'] == sample_team) 
]
print(example[['team', 'outcome', 'prob_win']])
print(f"Sum: {example['prob_win'].sum():.6f}")

# Save to CSV
double_test_df["line"] = 5.5
double_test_df.to_csv('test_predictions_v4.csv', index=False)
print(f"\nSaved {len(double_test_df)} test rows")

Model classes: [0 1]
Class 0: 0
Class 1: 1

Original y_train unique values:
[1 0]

Mapping:
0 = 'under'
1 = 'over'
'over' is at index: 1
'under' is at index: 0

Sample of double rows:
                                              match_url            team  \
3391  https://fbref.com/en/matches/000fb8fc/RB-Leipz...      RB Leipzig   
3392  https://fbref.com/en/matches/000fb8fc/RB-Leipz...   Holstein Kiel   
8613  https://fbref.com/en/matches/000fb8fc/RB-Leipz...      RB Leipzig   
8614  https://fbref.com/en/matches/000fb8fc/RB-Leipz...   Holstein Kiel   
2032  https://fbref.com/en/matches/001d3a5d/Lecce-Mi...           Lecce   
2033  https://fbref.com/en/matches/001d3a5d/Lecce-Mi...           Milan   
7254  https://fbref.com/en/matches/001d3a5d/Lecce-Mi...           Lecce   
7255  https://fbref.com/en/matches/001d3a5d/Lecce-Mi...           Milan   
3607  https://fbref.com/en/matches/005a65f4/Eintrach...  Eint Frankfurt   
3608  https://fbref.com/en/matches/005a65f4/Eintrach...        Dor

In [82]:
# Create dataframe with test set results
full_test_df = final_df.loc[X_test.index].copy()
full_test_df['predicted_is_won'] = test_predictions
full_test_df['prob_win'] = test_proba[:, 1]  # Probability of winning

In [83]:
double_test_df[(double_test_df["team"] == "Lecce") & (double_test_df["opp_team"] == "Milan") & (double_test_df["is_home"] == 1)]

,match_url,match_date,season,division,team,opp_team,is_home,gw,is_promoted?,is_early_season?,corner_kicks,outcome,predicted_is_won,prob_win,line
2032,https://fbref.com/en/matches/001d3a5d/Lecce-Mi...,2025-03-08,2024-2025,Serie A,Lecce,Milan,1,28,0,0,2.0,over,0,0.305828,5.5
7254,https://fbref.com/en/matches/001d3a5d/Lecce-Mi...,2025-03-08,2024-2025,Serie A,Lecce,Milan,1,28,0,0,2.0,under,1,0.694172,5.5


In [84]:
top_features = [
   "corner_kicks",
   "prob_win",
   "team_rolling_pass_types_corner_kicks",
   "team_rolling_misc_crosses", 
   "opp_team_rolling_pass_types_corner_kicks_out",
   "team_rolling_conceded_pass_types_through_balls",
   "team_rolling_pass_types_corner_kicks_out",
   "team_rolling_conceded_defense_errors",
   "team_rolling_conceded_misc_tackles_won",
   "opp_team_rolling_conceded_defense_errors",
   "team_rolling_conceded_possession_carries_into_penalty_area",
   "opp_team_rolling_misc_offsides",
   "team_rolling_summary_shots_on_target",
   "team_rolling_possession_touches_def_3rd",
   "team_rolling_possession_miscontrols",
   "team_rolling_is_early_season?",
   "team_rolling_conceded_misc_ball_recoveries",
   "opp_team_rolling_conceded_pass_types_corner_kicks",
   "team_rolling_summary_assists",
   "team_rolling_conceded_pass_types_corner_kicks_in",
   "team_rolling_conceded_defense_tackles_def_3rd",
   "opp_team_rolling_possession_dispossessed",
   "opp_team_rolling_conceded_possession_touches_def_3rd",
   "opp_team_rolling_conceded_pass_types_corner_kicks_out"
]

# Now use it with your filter
full_test_df[(full_test_df["team"] == "Arsenal") & 
            (full_test_df["opp_team"] == "Bournemouth") & 
            (full_test_df["is_home"] == 1)][top_features]

,corner_kicks,prob_win,team_rolling_pass_types_corner_kicks,team_rolling_misc_crosses,opp_team_rolling_pass_types_corner_kicks_out,team_rolling_conceded_pass_types_through_balls,team_rolling_pass_types_corner_kicks_out,team_rolling_conceded_defense_errors,team_rolling_conceded_misc_tackles_won,opp_team_rolling_conceded_defense_errors,...,team_rolling_possession_miscontrols,team_rolling_is_early_season?,team_rolling_conceded_misc_ball_recoveries,opp_team_rolling_conceded_pass_types_corner_kicks,team_rolling_summary_assists,team_rolling_conceded_pass_types_corner_kicks_in,team_rolling_conceded_defense_tackles_def_3rd,opp_team_rolling_possession_dispossessed,opp_team_rolling_conceded_possession_touches_def_3rd,opp_team_rolling_conceded_pass_types_corner_kicks_out
70,2.0,0.59402,7.079278,21.135962,0.51063,1.101185,0.019286,0.997192,11.565481,1.361833,...,12.811074,0.128566,34.443964,5.243003,1.477119,1.77642,10.904182,9.349866,215.650166,1.238347
